### Puzzle

https://adventofcode.com/2020/day/16

### Imports

In [1]:
import pandas as pd

### Load Input

In [2]:
# Store the location of the input directory
data_dir = '../../../data/2020'

# Open the input and store a list of each item as an int
with open(f"{data_dir}/day16_input.txt") as f:
    inputs = f.read().splitlines()

In [14]:
# Initialize a dictionary to store the ranges of each of the fields
field_dict = dict()

# Split the input and grab only the field ranges
field_ranges = inputs[:inputs.index('')]

# For each field, store a list with every possible value for that field
for field_range in field_ranges:
    field = field_range.split(':')[0]
    field = field.replace(' ', '_')
    
    ranges = field_range.split(': ')[1]
    
    low_range = ranges.split(' or ')[0]
    low_range_min = int(low_range.split('-')[0])
    low_range_max = int(low_range.split('-')[1]) + 1
    
    high_range = ranges.split(' or ')[1]
    high_range_min = int(high_range.split('-')[0])
    high_range_max = int(high_range.split('-')[1]) + 1
    
    field_dict[field] = list(range(low_range_min, low_range_max)) + list(range(high_range_min, high_range_max))

# Split the input and grab only your ticket and turn it into a list of ints
your_ticket = inputs[inputs.index('your ticket:') + 1]
your_ticket = [int(x) for x in your_ticket.split(',')]

# Split the input and grab only the nearby tickets and turn it into a list of lists of ints
nearby_tickets = inputs[inputs.index('nearby tickets:') + 1:]
nearby_tickets = [[int(x) for x in nearby_ticket.split(',')] for nearby_ticket in nearby_tickets]

### Part 1

In [20]:
# Turn the list of lists of all valid field values into a single list
all_valid_values = []
for valid_values in field_dict.values():
    all_valid_values += valid_values

# Initialize a counter for the error rate
error_rate = 0

# Check every field in every nearby ticket and add the value of the invalid field to the error_rate counter
for nearby_ticket in nearby_tickets:
    for field in nearby_ticket:
        if field not in all_valid_values:
            error_rate += field
            
print(error_rate)

23122


### Part 2

In [21]:
# Throw out all invalid tickets
# Keep a list of invalid tickets
invalid_tickets = []
for nearby_ticket in nearby_tickets:
    for field in nearby_ticket:
        # If any of the fields are invalid, add that ticket to the list of invalid tickets
        if field not in all_valid_values:
            invalid_tickets.append(nearby_ticket)
            break
            
# Get the list of valid tickets by removing invalid tickets
valid_tickets = [nearby_ticket for nearby_ticket in nearby_tickets if nearby_ticket not in invalid_tickets]

# Store the valid tickets in a dataframe
valid_tickets_df = pd.DataFrame(valid_tickets)

In [22]:
# Create a dictionary that stores the mapping of the field name to the index in the ticket field list
field_mapping_dict = dict()

# Initialize all values to be -1
for key in field_dict.keys():
    field_mapping_dict[key] = -1

In [23]:
# For each column, check to see if all values in that column fit in any of the ranges
possible_mappings = []
for col in valid_tickets_df.columns:
    for field_name, field_values in field_dict.items():
        if valid_tickets_df[col].isin(field_values).all():
            possible_mappings.append([col, field_name])  

# Store all possible column-field mappings in a dataframe
possible_mappings_df = pd.DataFrame(possible_mappings, columns=['index', 'field'])

In [24]:
# Iterate until all fields have been identified
while -1 in field_mapping_dict.values():
    # Orphans are column-field mappings where there is only mapping from column to field or field to column 
    orphans = []

    # If there is a column with only one valid field, that column must map to that field
    for index in possible_mappings_df['index'].unique():
        possible_fields = possible_mappings_df[possible_mappings_df['index'] == index]

        if len(possible_fields) == 1:
            orphans.append([possible_fields.iloc[0]['index'], possible_fields.iloc[0]['field']])

    # If there is a field with only one valid column, that column must map to that field
    for field in possible_mappings_df['field'].unique():
        possible_indices = possible_mappings_df[possible_mappings_df['field'] == field]

        if len(possible_indices) == 1:
            orphans.append([possible_indices.iloc[0]['index'], possible_indices.iloc[0]['field']])

    # Add each orphan to the column-field dictionary mapping
    for orphan in orphans:
        field_mapping_dict[orphan[1]] = orphan[0]
        possible_mappings_df = possible_mappings_df[possible_mappings_df['index'] != orphan[0]]
        possible_mappings_df = possible_mappings_df[possible_mappings_df['field'] != orphan[1]]

In [25]:
# Initialize a product to multiply my departure_ fields
total_product = 1

# For each column that is a departure field, multiply my ticket's value in that field to the running product
for key, value in field_mapping_dict.items():
    if key.startswith('departure'):
        total_product *= your_ticket[value]
        
print(total_product)

362974212989
